Padraic Flood, 13402438, 4BMS
# Description of the algorithm and design decisions
## The data

Given training data consisting of a vector, $\textbf{y}$, with elements $y^{(i)} \in \{0,1\}, i = 1, .. , N$ and a feature matrix $\textbf{X}$ with rows $(\textbf{x}^{i})^T$ , where the first column of $\textbf{X}$ is the **all ones column representing the intercept**. So $\textbf{X}$ has dimensions $N$ x $(M+1)$, where $M$ is number of features. $\textbf{X}$ must have **numerical entries**.
## The model - Logistic Regression
In logistic regression, with a coefficient vector $\theta$, we model the data using a **sigmoid function**, $h_\theta(x)$, as follows for an input $\textbf{x}$ and output $y$:

$$ P(y = 1 | \textbf{x}; \theta) = h_\theta(\textbf{x}) = \frac{1}{1 + exp(-\theta^T\textbf{x})} $$
and $$ P(y = 0 | \textbf{x}; \theta) = 1 - h_\theta(\textbf{x}) $$
combining these gives:
$$ P(y|\textbf{x};\theta) = h_\theta(\textbf{x})^{y}(1 - h_\theta(\textbf{x}))^{1-y}$$

## Maximum likelihood estimation of coefficients $\theta$
Assuming the data is **independent** we can calculate the likelihood of the data as follows:
$$L(\theta) = P(\textbf{y}|\textbf{X};\theta) = \prod_{i=1}^N P(y^{i}|\textbf{x}^i;\theta) = \prod_{i=1}^N h_\theta(\textbf{x}^{i})^{y^{i}}(1 - h_\theta(\textbf{x}^{i}))^{1-y^{i}} $$
Taking the log of this function makes it easier to differentiate and maximising the likelihood is the same as maximising the log likelihood, $l(\theta)$, since the log function is **monotonic increasing**
$$ l(\theta) = \log(L(\theta)) = \sum_{i=1}^N  [{y^{i}}\log h_\theta(\textbf{x}^{i}) + (1-y^{i})\log(1 - h_\theta(\textbf{x}^{i}))]$$
$$ l(\theta) = \sum_{i=1}^N  [{y^{i}}\log( \frac{h_\theta(\textbf{x}^{i})}{1-h_\theta(\textbf{x}^{i})}) + \log(\frac{1}{1 + exp(\theta^T\textbf{x})})]$$
$$ l(\theta) = \sum_{i=1}^N  [y^{i}\theta^T\textbf{x}^{i} - \log({1 + exp(\theta^T\textbf{x}^{i})})]$$
taking the partial derivatives for $j = 0,1, .., M$, where M is the number of features of $\textbf{X}$:
$$ \frac{\partial l(\theta)}{\partial \theta_j} = \sum_{i=1}^N  [y^{i}\textbf{x}_j^{i} - \frac{1}{1 + exp(\theta^T\textbf{x}^{i})} exp(\theta^T\textbf{x}^{i}) \textbf{x}_j^{i}] = \sum_{i=1}^N  [y^{i}\textbf{x}_j^{i} -  h_\theta(\textbf{x}^{i})\textbf{x}_j^{i}] = \sum_{i=1}^N  [y^{i} -  h_\theta(\textbf{x}^{i})]\textbf{x}^{i}_j$$
Or in vector-matrix product form:

let $\textbf{h}$ be the column vector with $\textbf{h}_i = h_\theta(\textbf{x}^i)$ for $i = 1, .. , N$

then the above partial derivatives, $\frac{\partial l(\theta)}{\partial \theta_j}$, can be rewritten as:
$$ \nabla l(\theta) = (\textbf{y} - \textbf{h})^T \textbf{X}$$

## Regularisation
To reduce overfitting I implemented regularisation of the coefficients $\theta$. I want to **maximise the log likelihood** of the training data but also **minimise the complexity** of the coefficients. Note: maximising the log likelihood is the same as minimising the negative of the log likelihood. So I want to minimise the following **cost function**:
$$ J(\theta) = -l(\theta) + \lambda \sum_{i=1}^M |\theta_i^q| $$
I have implemented q = 2 (penalty='l2'). But $q = 1$ (penalty='l1') gives a non-differentiable cost function which is harder to implement. lambda is a python keywords so I have decided instead use the scikit convention of using the parameter C for complexity and then setting $\lambda$ as $\lambda = \frac{1}{C}$. So a higher C means a higher model complexity (lower regularisation).

Note: conventionally this is divided by the number of samples but here I have chosen not to. The two methods are equivalent under a scaling of parameters (learning rate and lambda)

## Gradient descent
It can be shown that this is a convex function of $\theta$ and so we can minimise it using gradient descent. For q = 2:
$$ \frac{\partial J(\theta)}{\partial \theta_j} = - \frac{\partial l(\theta)}{\partial \theta_j} + \frac{2}{C}\theta_j$$
for $j=1,..,M$ and for the intercept (which we don't regularise):
$$\frac{\partial J(\theta)}{\partial \theta_0} = - \frac{\partial l(\theta)}{\partial \theta_0}$$
Then gradient descent repeats:
$$ \theta^{t+1} = \theta^{t} - \alpha \nabla J(\theta^t)$$
where $\alpha$ is the learning rate and $\nabla J(\theta)$ means the gradient of the cost function (i.e the vector of partial derivatives).
This repeats until convergence measured either by a max number of iterations or by the change in the cost function being under a certain tolerance.

## Newton's method.
In this case the cost function is convex and so finding the the global minimum is equivalent to finding a local minimum. This can be done by finding a point with zero gradient. Then convexity ensures it is a minimum and not a maximum. Finding a **root of the gradient function** can be done with newton's method, which in the multidimensional case is given by:
$$ \theta^{t+1} = \theta^{t} - H^{-1} \nabla J(\theta^t) $$
where H is the hessian matrix of $J(\theta)$
$$H_{ij} = \frac{\partial^2 J(\theta)}{\partial \theta_i \partial \theta_j} $$

$$ \frac{\partial l}{\partial \theta_j} = \sum_{k=1}^N  y^k\textbf{x}_j^k - \sum_{k=1}^N h_\theta(\textbf{x}^k)\textbf{x}_j^k$$
$$ \frac{\partial^2 l}{\partial \theta_i \partial \theta_j} = 0 -  \sum_{k=1}^N \frac{\partial }{\partial \theta_i} (h_\theta(\textbf{x}^k))\textbf{x}_j^k$$
$$ \frac{\partial^2 l}{\partial \theta_i \partial \theta_j} = -  \sum_{k=1}^N  (1-h_\theta(\textbf{x}^k))(h_\theta(\textbf{x}^k))\textbf{x}_i^k\textbf{x}_j^k$$
adding regularisation:
$$ H(J) =  - H(l) + \frac{2}{C}I_{M+1}$$
where I in the identity matrix, $H(l)$ is the hessian of $l(\theta)$, $H(J)$ is the hessian of $J(\theta)$ used in newton method's update equation above.

Newton's method takes far fewer iterations to converge than Gradient Descent. Its derivation using a Taylor expansion can be used to show it has **asymptotic quadratic convergence**. Also, due to the convexity of the cost function, it is guaranteed to converge. However, it requires inverting the Hessian matrix which can be an expensive operation if there is a very large number of features. There are heuristics to avoid this problem such as only inverting the hessian on a few iterations (maybe only once) and then trying to approximate the new inverse on subsequent iterations. Or you could try to approximate H by a diagonal matrix. Then inverting a diagonal matrix requires only taking the reciprocal of the diagonal entries. However I have not implemented these methods here.

## Multiclass Logistic regression
Often we are modelling more than two classes. To model this using logistic regression I implemented one-vs-rest logistic regression. Given $k$ target classes I model this as $k$ separate binary classification problem. So the $j$th class is set to 1 and all other classes are set to 0, for $j = 1,..,k$. Fitting a binary logistic model on this gives $\theta_j$. For a new test case, $\textbf{x}_{test}$, I calculate the probabilities $P(y_{test} = j | \textbf{x}_{test}; \theta_j) = h_{\theta_j}(x_{test})$ for $j = 1,..,k$. If all misclassification costs are equal we can then predict $\hat{y} = \underset{j = 1,..,k}{\operatorname{argmax}} h_{\theta_j}(x_{test})$

## Implementation
I used a similar basic API to Scikit to make it familiar to a new user and to allow it to work with other scikit functions and pipelines. This includes the same parameter names (tol, max\_iter, C, penalty..) as well as some variable names (coef\_, intercept\_, classes\_) and the scikit [convention](https://github.com/rasbt/python-machine-learning-book/blob/master/faq/underscore-convention.md) of using an underscore at the end of a variable name to indicate that it is estimated from the data. Note: I have NOT looked at the actual source code of scikit estimators, I am just attempting to keep it consistent with their public APIs.
* The data can be read in with the function **read_data**
* I implemented the function train_test_split to split the data
* I implement feature scaling with the function **standardise**. This should speed up convergence for gradient descent. Although I did not notice any effect on the small dataset I tested it on.
* The model parameters are passed to the constructor of LogisticRegression. The model is fit to training data with the **fit** method. Predictions on test data are found with the **predict** method. And probability estimates are gotten with the **predict_proba** method.
* My algorithm only supports numerical attributes. Although categorical attributes could be first converted to numerical representations using the tools provided in scikit

# Test results
* Both newton's method and gradient descent got average **accuracies above 95%** on the test sets of "owls.csv" with 10 random splits. I set the parameter C manually which might lead to a slight bias if I tried too many C values. A validation set could be used to avoid this.
* They achieved similarly high accuracies on the iris dataset.
* Full detailed results are **saved to the file "output"** and at the bottom of this PDF.

# Observations and conclusions
* **Newton's method** took **far fewer iterations** than gradient descent. Also since the Hessian matrix was only a 4x4 matrix in this case inverting it was not computationally intense and so newton's method was **much faster** than gradient descent. For a dataset with thousands of features it might be more efficient to use stochastic gradient descent.
* The lack of need to set the learning rate for newton's method makes it more convenient 
* With newton's method the coefficients sometimes started to explode in size. I think this is caused by perfectly separable training data which causes the optimal fit to approach a step function. This leads to a singular (not invertible) Hessian matrix. I avoided this with **L2 regularisation** and by setting a smaller max_iter

# Sources
* andrew ng's stanford machine learning course: https://www.youtube.com/watch?v=nLKOQfKLUks for newton's method and
https://www.youtube.com/watch?v=HZ4cvaztQEs for gradient descent for some details. But all the implementation and documentation here is entirely my own.

In [1]:
import pandas as pd
import numpy as np

class LogisticRegression:
    
    def __init__(self, max_iter=10000, learning_rate=.01, penalty='l2',
                 tol=1e-4, C=1.0, fit_method='gradient_descent'):
        self.max_iter = max_iter
        self.learning_rate = learning_rate
        self.C = C
        self.penalty = penalty
        self.tol = tol
        self.fit_method = fit_method
        
    def _call_fit_algorithm(self, X, y, index=0):
        """Fit coefficients with gradient descent or newton's method
        index: the class being fit in multiclass case (0 for binary case)
        """
        if self.fit_method == 'gradient_descent':
            self._gradient_descent(X, y, index)
        elif self.fit_method == 'newton':
            self._newtons_method(X, y, index)
        else:
            raise ValueError('error incorrect fit_method')
            
    def fit(self, X, y):
        """Fit the coefficents using training data X, y. 
        X: 2D numpy array with numerical attributes as columns 
           and samples as rows
        y: numpy array of labels"""
        self.classes_ = np.unique(y)
        self.num_classes_ = len(self.classes_)
        self.multi_class = self.num_classes_ > 2
        # Add a column of ones to X for the intercept:
        X = np.c_[np.ones(len(X)),X]
        
        # Multi-class case:
        if self.multi_class:
            # Initialise coefficients to zero
            self.theta_ = np.zeros((self.num_classes_, X.shape[1]))
            # Track the number of iterations taken for each class
            self.n_iter = np.zeros(self.num_classes_, dtype=np.uint32)
            # Fit a logistic regression model for each class
            for i, c in enumerate(self.classes_):
                # 1's for this class and 0's for all others
                y_c = 1*(y == c)
                self._call_fit_algorithm(X, y_c, i)
        
        # Binary case:
        else:
            # Default second class to positive label
            y_01 = 1*(y == self.classes_[1])
            # Make the coefficients 2D array to be consistent with multi-class.
            self.theta_ = np.zeros((1, X.shape[1]))
            # An array to be consistent with multi_class case. 
            # Make it an integer to improve asthetics of output.
            self.n_iter = np.zeros(1, dtype=np.uint32)
            self._call_fit_algorithm(X, y_01)
        
        # Set parameters names to those used by scikit 
        # to make it familiar to new user.
        self.intercept_ = self.theta_[:,0]
        self.coef_ = self.theta_[:,1:]
        
    def predict(self, X):
        """Return numpy array of predicted classes for test set X"""
        prob = self.predict_proba(X)
        # Find the index of the class with the maximum probability estimate 
        # for each test sample
        most_probable_indices = np.argmax(prob, axis = 1)
        # Convert from indices back to original class names
        most_probable_classes = self.classes_[most_probable_indices]
        return most_probable_classes
    
    def predict_proba(self, X):
        """Return 2D numpy array of probabilities for test set X
        The classes are given by the columns 
        and the test samples are given by the rows.
        """
        # Add a column of ones to X for the intercept:
        X = np.c_[np.ones(len(X)),X]
        try:
            prob = self._h(0, X)
            if self.multi_class:
                for i in range(1, self.num_classes_):
                    prob = np.c_[prob, self._h(i, X)]
            else:
                # Binary case
                # Note: I chose to model the second class as the positive label.
                prob = np.c_[1 - prob, prob]
            return prob
        except AttributeError:
            print("Error: The model must first be fit using the fit method")
        
    def _newtons_method(self, X, y, index):
        """Fits binary logistic model for class given by index. 
        Stores the coefficients in self.theta_[index].
        y must be in 0,1 form
        index: the class being fit in multiclass case (0 for binary case)
        """
        while self.n_iter[index] < self.max_iter:
            old_cost = self._cost(X, y, index)
            h = self._h(index, X)
            # Calculate the Hessian matrix
            H = np.zeros((X.shape[1],X.shape[1]))
            for k, x_k in enumerate(X):
                H += (1-h[k])*h[k]*np.outer(x_k, x_k)
            if self.penalty == 'l2':
                H += (2.0/self.C) * np.identity(X.shape[1])
            # Invert the Hessian
            try:
                H_inv = np.linalg.inv(H)
            except np.linalg.LinAlgError:
                # Sometimes the coefficients explode to infinity 
                # due to perfect seperation of the data.
                # This can be handled by more aggressive L2 regularization 
                # or by setting a lower max_iter.
                raise RuntimeError('Convergence error')
            # Apply newton's method to update coefficients
            self.theta_[index] -= H_inv.dot(self._grad_cost(X,y,index))
            
            # STOP if converged
            new_cost = self._cost(X, y, index)
            if abs(new_cost - old_cost) < self.tol:
                break
            self.n_iter[index] += 1

    def _gradient_descent(self, X, y, index):
        """Fits binary logistic model for class given by index. 
        Stores the coefficients in self.theta_.
        y must be in 0,1 form
        index: the class being fit in multiclass case (0 for binary case)
        """
        while self.n_iter[index] < self.max_iter:
            old_cost = self._cost(X, y, index)
            self.theta_[index] -= self.learning_rate *\
                                    self._grad_cost(X, y, index)
            new_cost = self._cost(X, y, index)
            # STOP if convergence is met.
            if abs(new_cost - old_cost) < self.tol:
                break
            self.n_iter += 1
            
    def _h(self, index, X):
        """Hypothesis function for current coefficients values.
        Returns numpy array of probability of the class, given by index, 
        for each row of X
        index: the class being fit in multiclass case (0 for binary case)"""
        return 1.0/(1 + np.exp(-X.dot(self.theta_[index])))
    
    def _cost(self, X, y, index):
        """Return the cost function J(theta) = -l(theta) + (regularization)
        y must be in 0,1 form
        index: the class being fit in multiclass case (0 for binary case)
        """
        h = self._h(index, X)
        return -(np.sum(y.dot(np.log(h)) + (1-y).dot(np.log(1-h)))) +\
                self._complexity(index)
    
    def _grad_cost(self, X, y, index):
        """Return the gradient of the cost function"""
        error = -(y - self._h(index, X))
        if self.penalty == 'l2':
            # Do not regularize the intercept
            return error.dot(X) + (1.0/self.C) * 2 *\
                    np.insert((1.0/self.C) * 2 * self.theta_[index,1:],0,0)
        # L1 regularization
        #elif self.penalty == 'l1':
        #    print 'l1 penalty not yet implemented'
        # No regularization
        elif self.penalty is None:
            return error.dot(X)
        else:
            print "Error: penalty must be either 'l1' or 'l2' or None"
            
    def _complexity(self, index):
        """Return the measure of the complexity of the model 
        under different regularization conditions.
        index: the class being fit in multiclass case (0 for binary case)"""
        if self.penalty == 'l2':
            return (1.0/self.C) * np.sum(np.square(self.theta_[index,1:])) 
        elif self.penalty == 'l1':
            return (1.0/self.C) * np.sum(np.abs(self.theta_[index,1:])) 
        elif self.penalty is None:
            return 0
        else:
            raise ValueError("Error: penalty must be either \
                                'l1' or 'l2' or None")


def read_data(filename, target, contains_header=True, attribute_names=None):
    """File must be a csv.
    contains_header: True if header is in file as first row.
    target: The name of the column representing the target 
            (output, dependent variable) of the model.
    Returns tuple of (X, y) where X is the feature matrix and y is labels."""
    if contains_header == False:
        d = pd.read_csv(filename, header=None, names=attribute_names)
    else:
        d = pd.read_csv(filename)
    y = d[target]
    X = d.drop(target, axis=1)
    # convert from pandas dataframe to numpy array
    X = X.values
    y = y.values
    return (X, y)

def standardise(X):
    """standardise columns of X to give mean 0, variance 1."""
    return(X - np.mean(X, axis=0))/np.std(X, axis=0)

def train_test_split(X, y, test_size):
    """test_size: fraction to put in test set"""
    p = np.random.permutation(len(y))
    split_index = int(round(len(y)*test_size))
    test = p[:split_index]
    train = p[split_index:]
    return (X[train], X[test], y[train], y[test])

## Testing

In [2]:
# INPUT file here
X, y = read_data('owls15.csv', contains_header=False, 
                 attribute_names = ['body-length', 'wing-length', 'body-width',
                                    'wing-width', 'type'],
                 target = 'type')
X = standardise(X)

test_scores = np.array([])
train_scores = np.array([])
out = open('output','w')
for i in range(10):
    out.write('Split ' + str(i+1) + '\n')
    (X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size = .33)
    logit = LogisticRegression(C=10, tol=1e-2, penalty='l2',
                               fit_method='newton', max_iter=100)
    #logit = LogisticRegression(C=15, tol=1e-3, penalty=None, max_iter=50000,
    #                          fit_method='gradient_descent', learning_rate=.05)
    logit.fit(X_train, y_train)
    #print logit.n_iter
    #print logit.coef_
    pred_test = logit.predict(X_test)
    pred_train = logit.predict(X_train)
    
    out.write('Actual labels\n')
    y_test.tofile(out, sep=',')
    out.write('\nPredicted labels\n')
    pred_test.tofile(out, sep=',')
    
    test_score = np.mean(y_test == pred_test)
    train_score = np.mean(y_train == pred_train)
    out.write('\nAccuracy: ' + str(int(100*test_score)) + '%\n')
    test_scores = np.r_[test_scores, test_score]
    train_scores = np.r_[train_scores, train_score]
    
print 'train accuracies', train_scores.round(2)
print 'test accuracies', test_scores.round(2)
print 'mean training accuracy %.02f, standard deviation %.02f' %\
        (np.mean(train_scores), np.std(train_scores))
print 'mean test accuracy %.02f, standard deviation %.02f' %\
        (np.mean(test_scores), np.std(test_scores))
out.write('\nmean test accuracy %.02f, standard deviation %.02f' %\
          (np.mean(test_scores), np.std(test_scores)))
out.close()

train accuracies [ 0.97  0.94  1.    0.96  0.98  0.97  0.99  0.98  0.96  0.96]
test accuracies [ 0.96  1.    0.93  0.98  0.91  0.93  0.91  0.93  0.98  1.  ]
mean training accuracy 0.97, standard deviation 0.02
mean test accuracy 0.95, standard deviation 0.03
